# Task2：正则关键词与文本分类
正则表达式（Regular Expressions，简称regex）是一种用于字符串搜索和操作的强大工具。它使用单个字符串来描述、匹配一系列符合某个句法规则的字符串。正则表达式在计算机科学、编程、数据挖掘和文本处理中有着广泛的应用。

- **定义规则**：根据分类需求，定义一组正则表达式规则。
- **预处理文本**：对输入文本进行清洗，如去除标点符号、转换为小写等。
- **模式匹配**：使用正则表达式在文本中搜索定义的模式。
- **分类决策**：根据匹配结果，将文本分配到相应的类别。

使用正则表达式进行文本分类时，确定关键词是一个关键步骤，因为它直接影响到分类的准确性和效率。可以从分析中找出每个类别的高频词汇，或考虑类别相关的专业术语或行业特定的词汇。

In [2]:
# 数据加载
# 读取数据集，这里是直接联网读取，也可以通过下载文件，再读取
import pandas as pd

data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

#  统计训练集的Intent分布
print(train_data[1].value_counts())
print(train_data[1].unique())
# 统计训练集的文本长度分布
print(train_data[0].apply(len).describe())

1
FilmTele-Play            1355
Video-Play               1334
Music-Play               1304
Radio-Listen             1285
Alarm-Update             1264
Weather-Query            1229
Travel-Query             1220
HomeAppliance-Control    1215
Calendar-Query           1214
TVProgram-Play            240
Audio-Play                226
Other                     214
Name: count, dtype: int64
['Travel-Query' 'Music-Play' 'FilmTele-Play' 'Video-Play' 'Radio-Listen'
 'HomeAppliance-Control' 'Weather-Query' 'Alarm-Update' 'Calendar-Query'
 'TVProgram-Play' 'Audio-Play' 'Other']
count    12100.000000
mean        15.138678
std          4.872583
min          4.000000
25%         12.000000
50%         15.000000
75%         18.000000
max         54.000000
Name: 0, dtype: float64


In [7]:
# 关键词词典构建
from collections import Counter 
import re 
import string 
import jieba 

def to_patterns(data):
    content = ''.join(list(data[0]))
    # 使用正则表达式去除所有空格、换行符和标点符号
    # 将所有的标点符号加入到正则表达式中
    punctuation = re.escape(string.punctuation) + "，。" # 对特殊字符进行转义

    # 创建正则表达式以匹配所有空格、换行符和标点符号
    content = re.sub(f'[{punctuation}\s]', '', content)

    words = jieba.lcut(content) 
    # 读取中文停用词
    cn_stopwords = ''.join(pd.read_csv('https://mirror.coggle.club/stopwords/baidu_stopwords.txt', header=None)[0])
    words = [x for x in words if x not in cn_stopwords]
    words = [x for x in words if len(x)>1]       
    words = [x for x in words if not x.isdigit()]
    
    word_counts = Counter(words)
    
    train_words = [x for x in words if word_counts[x]>5]  
    return train_words  
     
train_words = to_patterns(train_data)

# 构建词频词典
train_words_prior = {} 

for row in train_data.iloc[:].itertuples():    
    text , label  = row[1] , row[2]
    words = jieba.lcut(text)
    words = [x for x in words if x in train_words]  
    
    if len(words) == 0 : 
        continue 
    
    for word in words : 
        if word not in train_words_prior : 
            train_words_prior[word] = {"total" : 0 }  
            
        if label not in train_words_prior[word] : 
            train_words_prior[word][label] = 0 
        
        train_words_prior[word]["total"] += 1 
        train_words_prior[word][label] += 1  

In [8]:
train_words_prior  = pd.DataFrame(train_words_prior).T 
train_words_prior.fillna(0,inplace= True) 

In [9]:
for category  in train_data[1].unique():
    train_words_prior[category] /= train_words_prior["total"]


In [10]:
train_words_prior["category"]  =train_words_prior.columns[1:][train_words_prior.values[:,1:].argmax(1) ] 
train_words_prior.groupby("category").apply(lambda x:list(x.index)) 

/var/folders/fx/x5xt5flx4_s_bhn2f7sx897r0000gn/T/ipykernel_1674/1497967922.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_words_prior.groupby("category").apply(lambda x:list(x.index))


category
Alarm-Update             [早上, 我定, 下午, 参加, 公司, 闹钟, 活动, 提醒, 创建, 周末, 上午, 取...
Audio-Play               [故事, 小说, 广播剧, 英文版, 岳云鹏, 爆笑, 相声, 有声, 俄语, 第五章, 郭...
Calendar-Query           [昨天, 农历, 我查, 星期, 几号, 告诉, 几月, 查查, 礼拜, 几是, 春节, 母...
FilmTele-Play            [播放, 古装, 爱情, 电视剧, 一个, 推理, 一会, 地方, 导演, 赵丽颖, 麻烦,...
HomeAppliance-Control    [空调, 客厅, 风速, 打开, 烤箱, 调高, 洗衣机, 停止, 工作, 模式, 可不可以...
Music-Play               [随便, 一首, 专辑, 单曲, 循环, 王菲, 钢琴曲, 随机, 治愈, 日语, 歌曲, ...
Other                                                 [永远, 电话, 老婆, 漫画, 有人]
Radio-Listen             [河南, 新闻广播, 新闻台, 交通, 广播电台, 七点, 中央, 电台, 都市, 之声, ...
TVProgram-Play           [播出, 卫视, 广西, 法治, CCTV11, 剧场, 开播, 文化, 结束, 早间, 贵...
Travel-Query             [汽车票, 回家, 深圳, 武汉, 北京, 桂林, 飞机, 起飞, 快点, 三张, 成都, ...
Video-Play               [挑战, 游戏, 视频, 和平, 精英, 花絮, 转播, 比赛, 现场, 世界, 年谍, 第...
Weather-Query            [查询, 海南, 刮风, 几天, 山西, 明天, 衡水, 气温, 适合, 杭州, 香港, 紫...
dtype: object